In [ ]:
#!pip install datasets transformers -q

In [ ]:
#!pip install fsspec==2023.6.0 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 8.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2023.6.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-runtime-cu12==12.4.127; platform_system == "Linux" a

In [ ]:
from datasets import load_dataset

dataset = load_dataset("amazon_polarity")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [ ]:
small_train = dataset['train'].shuffle(seed=42).select(range(1000))
small_test = dataset['test'].shuffle(seed=42).select(range(50))

In [ ]:
import pandas as pd

df_train = pd.DataFrame(small_train)
df_test = pd.DataFrame(small_test)

df_train.head()

,label,title,content
0,0,Anyone who likes this better than the Pekinpah...,All the pretty people in this film. Even the R...
1,0,Author seems mentally unstable,I know that Tom Robbins has a loyal following ...
2,1,Spaetzle Noodles,This type of spaetzle maker is easier to manuv...
3,1,Cool product until the cats lost it . . . .,My cats seemed to be amused to work for their ...
4,1,Works as expected,I've had no problems running this with my Dell...


In [ ]:
df_train.describe(include='object')

,title,content
count,1000,1000
unique,987,1000
top,Excellent,This flick made me angry and sad but also made...
freq,4,1


In [ ]:
from datasets import Dataset, DatasetDict

train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)

train_dataset, test_dataset

(Dataset({
     features: ['label', 'title', 'content'],
     num_rows: 1000
 }),
 Dataset({
     features: ['label', 'title', 'content'],
     num_rows: 50
 }))

In [ ]:
model_nm = 'distilbert-base-uncased'

In [ ]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
tokz = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
def tok_func(batch):
    # Combine title and content for each example in the batch
    combined_texts = [t + " " + c for t, c in zip(batch["title"], batch["content"])]
    return tokz(combined_texts, truncation=True, padding='max_length', max_length=128)

In [ ]:
tok_ds = train_dataset.map(tok_func, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
dds = tok_ds.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['label', 'title', 'content', 'input_ids', 'attention_mask'],
        num_rows: 750
    })
    test: Dataset({
        features: ['label', 'title', 'content', 'input_ids', 'attention_mask'],
        num_rows: 250
    })
})

In [ ]:
eval_ds = test_dataset.map(tok_func, batched=True)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments,Trainer

In [ ]:
# Define batch_size, epochs and learning_rate
bs = 8
epochs = 3
lr = 2e-5

In [ ]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
                         per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2, num_train_epochs=epochs,
                         weight_decay=0.01, report_to='none', eval_strategy='epoch', save_strategy='epoch')

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

def compute_metrics(eval_pred):
  logits, labels = eval_pred
  preds = np.argmax(logits, axis=-1)
  return {
      "accuracy": accuracy_score(labels, preds),
      "f1": f1_score(labels, preds)
  }

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=2)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz, compute_metrics=compute_metrics)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-23-2529755874.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.367329,0.836000,0.840467
2,No log,0.374949,0.876000,0.870293


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.367329,0.836000,0.840467
2,No log,0.374949,0.876000,0.870293
3,No log,0.400300,0.872000,0.865546


TrainOutput(global_step=282, training_loss=0.2560543033248144, metrics={'train_runtime': 2088.8471, 'train_samples_per_second': 1.077, 'train_steps_per_second': 0.135, 'total_flos': 74512911744000.0, 'train_loss': 0.2560543033248144, 'epoch': 3.0})

In [ ]:
trainer.save_model("./amazon-polarity-model")  # Save model and tokenizer
tokz.save_pretrained("./amazon-polarity-model")

('./amazon-polarity-model/tokenizer_config.json',
 './amazon-polarity-model/special_tokens_map.json',
 './amazon-polarity-model/vocab.txt',
 './amazon-polarity-model/added_tokens.json',
 './amazon-polarity-model/tokenizer.json')

In [ ]:
preds = trainer.predict(eval_ds).predictions.argmax(axis=-1)
print(preds[:10])

[0 0 0 0 0 1 1 1 0 1]


In [ ]:
def classify_review(title, content):
    inputs = tokz(title + " " + content, return_tensors="pt", truncation=True, padding="max_length", max_length=128)
    outputs = model(**inputs)
    probs = outputs.logits.softmax(dim=1)
    pred = probs.argmax().item()
    label = "Positive" if pred == 1 else "Negative"
    return label, probs.detach().numpy()

# Example
classify_review(
    title="A must-watch!",
    content="This movie was absolutely stunning. Great acting and direction."
)

('Positive', array([[0.00947008, 0.99052995]], dtype=float32))